In [123]:
import os

In [124]:
%pwd

'C:\\Users\\ba415\\OneDrive\\Desktop\\ENDTOEND-REALESTATE-MLPROJECT'

In [125]:
os.chdir('../')

In [126]:
%pwd

'C:\\Users\\ba415\\OneDrive\\Desktop'

In [127]:
import os
os.chdir('C:/Users/ba415/OneDrive/Desktop/ENDTOEND-REALESTATE-MLPROJECT')
print(os.getcwd())  # This should print 'C:/Users/ba415/OneDrive/Desktop/ENDTOEND-REALESTATE-MLPROJECT'

C:\Users\ba415\OneDrive\Desktop\ENDTOEND-REALESTATE-MLPROJECT


In [128]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('__file__')), '..', 'src')))


In [129]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories
from src.mlProject import logger

In [130]:
from src.mlProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.mlProject.utils.common import read_yaml, create_directories
from src.mlProject import logger


In [131]:
# making entity -> defining the return type for a funtion
# Define DataIngestionConfig class
from dataclasses import dataclass

@dataclass
class DataIngestionConfig:
    root_dir: str
    source_URL: str
    local_data_file: str
    unzip_dir: str

In [132]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config_filepath = config_filepath
        self.params_filepath = params_filepath
        self.schema_filepath = schema_filepath

        logger.info(f"Using config file at: {self.config_filepath}")
        logger.info(f"Using params file at: {self.params_filepath}")
        logger.info(f"Using schema file at: {self.schema_filepath}")

        self.config = read_yaml(self.config_filepath)
        self.params = read_yaml(self.params_filepath)
        self.schema = read_yaml(self.schema_filepath)
        
        logger.info(f"Configuration loaded: {self.config}")

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.get('data_ingestion', {})
        create_directories([config.get('root_dir', 'artifacts/data_ingestion')])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.get('root_dir', 'artifacts/data_ingestion'),
            source_URL=config.get('source_URL', ''),
            local_data_file=config.get('local_data_file', ''),
            unzip_dir=config.get('unzip_dir', '')
        )
        
        return data_ingestion_config

In [133]:
import zipfile
from urllib import request
import requests
import filetype

In [134]:
class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_URL)
            with open(self.config.local_data_file, 'wb') as file:
                file.write(response.content)
            logger.info(f"File downloaded: {self.config.local_data_file}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file}")

    def extract_zip_file(self):
        if zipfile.is_zipfile(self.config.local_data_file):
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)
            logger.info(f"File extracted to: {self.config.unzip_dir}")
        else:
            logger.error("Downloaded file is not a zip file.")

In [135]:
# Main block
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(e)

[2024-07-31 04:55:59,883: INFO: 639775234: Using config file at: config\config.yaml]
[2024-07-31 04:55:59,883: INFO: 639775234: Using params file at: params.yaml]
[2024-07-31 04:55:59,884: INFO: 639775234: Using schema file at: schema.yaml]
[2024-07-31 04:55:59,886: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-31 04:55:59,887: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-31 04:55:59,889: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-31 04:55:59,890: INFO: 639775234: Configuration loaded: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/Lazygeniusbilal/Dataset/raw/main/graana_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion/unzipped'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/'}}]
[2024-07-31 04:55:59,891: INFO: common: cr

In [137]:
# data ingestion done